In [71]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timezone

ROUNDSTATS_CSV   = "data/vlr_roundstats_demo.csv"
PLAYER_STATS_CSV = "data/vlr_playerstats_demo.csv"
MATCHDATA_CSV    = "data/vlr_matchstats_demo.csv"
OUT_PATH         = "output/vlr_patchpool_demo.csv"


roundstats = pd.read_csv(ROUNDSTATS_CSV)
player_stats = pd.read_csv(PLAYER_STATS_CSV)
df_matchdata = pd.read_csv(MATCHDATA_CSV)

#converts wide roundstats (both teams per row) into long team-level rows, enabling clean Round_Diff aggregation later
team_rounds = []
for _, row in roundstats.iterrows():
    team_rounds.extend([
        {'game_id': row['game_id'], 'team_num': 1, 'result': row['team1_result']},
        {'game_id': row['game_id'], 'team_num': 2, 'result': row['team2_result']},
    ])

#create team-level round df; drop invalid ids, add binary win indicator
round_df = pd.DataFrame(team_rounds)
round_df['is_win'] = (round_df['result'] == 'won').astype(int)

#group by game_id/team_num to count total rounds/wins, then calculate per-team round diff
total_rounds = round_df.groupby(['game_id', 'team_num']).size().reset_index(name='total_rounds')
wins_by_team = round_df.groupby(['game_id', 'team_num'])['is_win'].sum().reset_index(name='wins')
round_diff_lookup = pd.merge(total_rounds, wins_by_team, on=['game_id', 'team_num'])
round_diff_lookup['Round_Diff'] = 2 * round_diff_lookup['wins'] - round_diff_lookup['total_rounds']

#nested lookup to retrieve individual team round diffs at a game_id level
rd_lookup = {}
for _, r in round_diff_lookup.iterrows():
    rd_lookup.setdefault(r['game_id'], {})[r['team_num']] = r['Round_Diff']

#converts match_datetime into datetime64 objects; error handling/force to UTC
df_matchdata['match_datetime'] = pd.to_datetime(df_matchdata['match_datetime'], utc=True, errors='coerce')
df_matchdata['game_id'] = pd.to_numeric(df_matchdata['game_id'], errors='coerce').astype('Int64')

pb_cols = [f'picks_and_bans.{i}' for i in range(1, 8)]
for i in range(1, 8):
    df_matchdata[f'picks_and_bans.{i}'] = None

#picks and bans parsing, stripping brackets/quotes, splitting commas, returning 7 elements that comprise a match-level pick and ban process
def parse_picks_bans(pb_string):
    if pd.isna(pb_string):
        return [None] * 7
    s = str(pb_string).strip('{}').replace("'", "")
    items = [item.strip() for item in s.split(',')]
    return (items + [None] * 7)[:7]

#applies parsing, expands picks and bans into columns
pb_parsed = df_matchdata['picks_and_bans'].apply(parse_picks_bans).tolist()
for i, col in enumerate(pb_cols):
    df_matchdata[col] = [row[i] for row in pb_parsed]

#define series grouping keys, build matchid counter
series_groups = df_matchdata.groupby([
    'match_datetime', 'team1_name', 'team2_name', 'competition_name', 'match_best_of'
])

series_records = []
match_id_counter = 1

#for each series group: order maps by game_id, collect map ids, attach picks and bans, and subsequently create matchid
for group_key, group_df in series_groups:
    unique_games = sorted(group_df['game_id'].dropna().astype(int).unique())
    
    if len(unique_games) > 1:
        start = unique_games[0]
        expected = list(range(start, start + len(unique_games)))
        if unique_games != expected:
            continue
    
    first_game = group_df.iloc[0]
    patch_mode = group_df['match_patch'].mode()
    match_patch = patch_mode.iat[0] if not patch_mode.empty else first_game['match_patch']
    
    map_ids = {}
    for i in range(1, 6):
        map_ids[f'Map{i}_ID'] = unique_games[i-1] if i <= len(unique_games) else None
    
    pb_data = {col: first_game[col] for col in pb_cols}
    
    team1, team2 = group_key[1], group_key[2]
    
    for team_name in [team1, team2]:
        team_num = 1 if team_name == team1 else 2
        
        map_rds = {}
        for i in range(1, 6):
            mid = map_ids[f'Map{i}_ID']
            if pd.notna(mid) and int(mid) in rd_lookup and team_num in rd_lookup[int(mid)]:
                map_rds[f'Map{i}_RD'] = rd_lookup[int(mid)][team_num]
            else:
                map_rds[f'Map{i}_RD'] = pd.NA
        
        sum_map_rd = sum(v for v in map_rds.values() if pd.notna(v))
        
        series_records.append({
            'MatchID': match_id_counter,
            'team': team_name,
            'team1_name': team1,
            'team2_name': team2,
            'series_datetime': group_key[0],
            'competition_name': group_key[3],
            'match_best_of': group_key[4],
            'match_patch': match_patch,
            'SeriesRoundDiff': sum_map_rd,
            **map_ids,
            **map_rds,
            **pb_data
        })
    
    match_id_counter += 1

df_series = pd.DataFrame(series_records)

for i in range(1, 6):
    df_series[f"Map{i}_ID"] = pd.to_numeric(df_series[f"Map{i}_ID"], errors='coerce')

df_series['team'] = df_series['team'].astype(str).str.strip()

player_stats['player_team_abbrev'] = player_stats['player_team'].str.extract(r'^([^\(]+)')[0].str.strip()
player_stats['player_team_full'] = player_stats['player_team'].str.extract(r'\(([^)]+)\)')[0].str.strip()

mapping = player_stats[['player_team_abbrev', 'player_team_full']].drop_duplicates()
full_to_all_abbrevs = mapping.groupby('player_team_full')['player_team_abbrev'].apply(set).to_dict()

def extract_pb_per_team(df, team_num):
    records = []
    team_col = f'team{team_num}_name'
    
    for idx, row in df.iterrows():
        known = full_to_all_abbrevs.get(row[team_col], set())
        for col in pb_cols:
            parts = str(row.get(col, '')).split(maxsplit=2)
            if len(parts) == 3:
                abbr, action, map_name = parts
                if abbr in known:
                    records.append({'idx': idx, 'team_num': team_num, 'action': action, 'map_name': map_name})
    
    return pd.DataFrame(records)

def pivot_pb(df, team_num, action):
    if len(df) == 0:
        return pd.DataFrame()
    
    sub = df[(df['team_num'] == team_num) & (df['action'] == action)].copy()
    if len(sub) == 0:
        return pd.DataFrame()
    
    sub['n'] = sub.groupby('idx').cumcount() + 1
    wide = sub.pivot(index='idx', columns='n', values='map_name')
    wide.columns = [f'team{team_num}_{action}_{i}' for i in wide.columns]
    return wide

pb_team1 = extract_pb_per_team(df_series, 1)
pb_team2 = extract_pb_per_team(df_series, 2)

p1_pick = pivot_pb(pb_team1, 1, 'pick')
p2_pick = pivot_pb(pb_team2, 2, 'pick')
p1_ban = pivot_pb(pb_team1, 1, 'ban')
p2_ban = pivot_pb(pb_team2, 2, 'ban')

for name, pivot_table in [('p1_pick', p1_pick), ('p1_ban', p1_ban), ('p2_pick', p2_pick), ('p2_ban', p2_ban)]:
    if len(pivot_table) > 0:
        overlap = set(pivot_table.columns).intersection(set(df_series.columns))
        if overlap:
            print(f"Dropping overlapping columns from {name}: {overlap}")
            df_series = df_series.drop(columns=list(overlap))
        df_series = df_series.join(pivot_table, how='left')

have = {'team1_pick_1','team2_pick_1'}.intersection(df_series.columns)
if have:
    df_series = df_series.groupby('Map1_ID').filter(
        lambda g: not all(g[c].isna().all() for c in have)
    )

df_final = pd.merge(
    df_series,
    player_stats[['game_id', 'player_team_abbrev', 'player_team_full']],
    left_on=['Map1_ID', 'team'],
    right_on=['game_id', 'player_team_full'],
    how='left',
    suffixes=('', '_ps')
).drop(columns=['game_id'])

df_final = df_final.drop_duplicates(subset=['MatchID', 'team'])

mapname_lkup = df_matchdata.set_index('game_id')['map_name'].to_dict()
for i in range(1, 6):
    df_final[f"Map{i}_name"] = df_final[f"Map{i}_ID"].map(mapname_lkup)

series_dt = df_matchdata[['game_id', 'match_datetime']].dropna().drop_duplicates().rename(
    columns={'game_id': 'series_id', 'match_datetime': 'series_datetime'}
)

series_roster = (
    player_stats.rename(columns={'game_id': 'series_id', 'player_team_full': 'team_full'})
    .groupby(['series_id', 'team_full'])['player_handle']
    .apply(lambda lst: sorted(set(lst)))
    .reset_index()
    .merge(series_dt, on='series_id', how='left')
)

def _canon_handle(s):
    return str(s).strip().lower()

def _to_utc(dt):
    if pd.isna(dt):
        return None
    if getattr(dt, "tzinfo", None) is None:
        return dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

ROSTER_CONTINUITY_RULES = [
    {"from_brand": "Envy", "to_brand": "OpTic Gaming", "effective_ts": datetime(2022, 2, 20, 21, 0, 0, tzinfo=timezone.utc)},
    {"from_brand": "Rankers", "to_brand": "Envy", "effective_ts": datetime(2025, 3, 10, 20, 0, 0, tzinfo=timezone.utc)},
]

BRAND_RESETS = {
    "Version1": [datetime(2023, 1, 10, 23, 0, 0, tzinfo=timezone.utc)],
}

forward_map = {}
reverse_map = {}

for r in ROSTER_CONTINUITY_RULES:
    f = r["from_brand"].strip()
    t = r["to_brand"].strip()
    ts = r["effective_ts"]
    forward_map.setdefault(f, []).append((t, ts))
    reverse_map.setdefault(t, []).append((f, ts))

def _era_for(brand: str, when_utc):
    if when_utc is None:
        return 1
    resets = BRAND_RESETS.get((brand or "").strip(), [])
    if not resets:
        return 1
    cnt = sum(1 for ts in resets if when_utc >= ts)
    return 1 + cnt

def continuity_key_for(brand: str, when):
    brand = (brand or "").strip()
    when_utc = _to_utc(when)
    
    def earliest_ancestor(b, ts):
        if b in reverse_map and ts is not None:
            candidates = [(src, eff) for (src, eff) in reverse_map[b] if ts >= eff]
            if candidates:
                src, eff = max(candidates, key=lambda x: x[1])
                return earliest_ancestor(src, ts)
        return b
    
    def chain_label(root):
        visited = set([root])
        segs = [root]
        cur = root
        while cur in forward_map:
            nxt, _ = sorted(forward_map[cur], key=lambda x: x[1])[0]
            if nxt in visited:
                break
            segs.append(nxt)
            visited.add(nxt)
            cur = nxt
        return "→".join(segs)
    
    anc = earliest_ancestor(brand, when_utc)
    label = chain_label(anc)
    era = _era_for(brand, when_utc)
    return f"CHAIN:{label} | ERA:{era}"

series_roster["series_datetime"] = pd.to_datetime(series_roster["series_datetime"], utc=True, errors="coerce")
series_roster["continuity_key"] = series_roster.apply(
    lambda r: continuity_key_for(r["team_full"], r["series_datetime"]),
    axis=1
)

# Apply disambiguation to df_final - need to get roster info first
# Get roster from player_stats merge
roster_lookup = (
    player_stats.rename(columns={'game_id': 'series_id'})
    .groupby(['series_id', 'player_team_full'])['player_handle']
    .apply(lambda lst: sorted(set(lst)))
    .reset_index()
)

# Merge roster into df_final
df_final = df_final.merge(
    roster_lookup.rename(columns={'series_id': 'Map1_ID', 'player_team_full': 'team'}),
    on=['Map1_ID', 'team'],
    how='left'
)

# Set team name columns
df_final['team_full_raw'] = df_final['team']
df_final['team_full_corr'] = df_final['team']

df_final["series_datetime"] = pd.to_datetime(df_final["series_datetime"], utc=True, errors="coerce")
df_final["continuity_key"] = df_final.apply(
    lambda r: continuity_key_for(r["team_full_corr"], r["series_datetime"]),
    axis=1
)

team_series = {
    key: grp.sort_values('series_datetime').reset_index(drop=True)
    for key, grp in series_roster.groupby('continuity_key', dropna=False)
}

def compute_days_since_fast(roster, current_date, k, cont_key):
    if not isinstance(roster, (list, tuple)) or pd.isna(current_date) or pd.isna(cont_key):
        return pd.NA
    if cont_key not in team_series:
        return pd.NA
    past = team_series[cont_key]
    past = past[past['series_datetime'] <= current_date]
    if past.empty:
        return pd.NA
    key = set(roster)
    mask = past['player_handle'].apply(lambda ph: len(set(ph).intersection(key)) >= k)
    if not mask.any():
        return 0
    first = past.loc[mask, 'series_datetime'].min()
    return int((current_date - first).total_seconds() // 86400)

for k in (3, 4, 5):
    df_final[f'days_since_{k}_of_5'] = df_final.apply(
        lambda r: compute_days_since_fast(
            r.get('player_handle'),
            r['series_datetime'],
            k,
            r['continuity_key']
        ),
        axis=1
    )

df_final = df_final.sort_values(['continuity_key', 'series_datetime'])
df_final['DaysSinceLastMatch'] = (
    df_final.groupby('continuity_key')['series_datetime']
    .diff()
    .dt.total_seconds()
    .div(86400)
    .astype('float')
)

df_final = df_final.drop(columns=['player_handle'], errors='ignore').drop_duplicates()

def parse_pb_cell(s):
    if not isinstance(s, str) or not s.strip():
        return None
    toks = s.split()
    if len(toks) >= 3 and toks[1].lower() in {'pick', 'ban'}:
        return toks[2]
    if 'remains' in s.lower():
        return toks[0] if toks else None
    if len(toks) == 1 and toks[0][0].isupper():
        return toks[0]
    return None

for col in pb_cols:
    if col in df_final.columns:
        df_final[col] = df_final[col].apply(parse_pb_cell)

df_final['pb_valid_count'] = df_final[pb_cols].apply(lambda row: sum(pd.notna(row)), axis=1)
df_final = df_final[df_final['pb_valid_count'] > 0].drop(columns='pb_valid_count')

df_final['num_real_picks'] = df_final[pb_cols].apply(
    lambda row: sum(isinstance(val, str) and val.lower() != 'remains' for val in row),
    axis=1
)
df_final = df_final[df_final['num_real_picks'] >= 2].drop(columns='num_real_picks')

df_final['MapSet'] = df_final[pb_cols].values.tolist()
df_final['MapSet'] = df_final['MapSet'].apply(lambda x: frozenset(filter(None, x)))

print("Rows before 7-map gate:", len(df_final))
df_final = df_final[df_final['MapSet'].apply(len) == 7]
print("Rows after 7-map gate:", len(df_final))

df_final = df_final.sort_values('series_datetime')
unique_mapsets = df_final['MapSet'].drop_duplicates()
total_pools = len(unique_mapsets)
pool_id_map = {ms: (total_pools - i) for i, ms in enumerate(unique_mapsets)}

df_final['PoolID'] = df_final['MapSet'].map(pool_id_map)
df_final['ActiveMapPool'] = df_final['MapSet'].apply(lambda x: sorted(list(x)))
df_final['ReversedPoolID'] = df_final['PoolID'].max() + 1 - df_final['PoolID']
df_final = df_final.drop(columns=['MapSet'])

print(f"PoolID gaps check: {sorted(df_final['PoolID'].unique())}")

df_final = df_final.sort_values(['series_datetime', 'team1_name', 'team2_name'])
match_id_remap = {old_id: new_id for new_id, old_id in enumerate(sorted(df_final['MatchID'].unique()), 1)}
df_final['MatchID'] = df_final['MatchID'].map(match_id_remap)

def clean_patch_string(patch):
    s = str(patch).strip()
    m = re.search(r'(\d+)\.(\d+)', s)
    return f"Patch {m.group(1)}.{m.group(2)}" if m else np.nan

def patch_to_index(patch):
    if pd.isna(patch):
        return np.nan
    s = str(patch).strip()
    m = re.search(r'(\d+)\.(\d+)', s)
    if not m:
        return np.nan
    major, minor = int(m.group(1)), int(m.group(2))
    return major * 100 + minor

df_final['match_patch'] = df_final['match_patch'].apply(clean_patch_string)
df_final['patch_index'] = df_final['match_patch'].apply(patch_to_index)
df_final['patch_major'] = df_final['match_patch'].astype(str).str.extract(r'(\d+)').astype('Int64')

nan_patches = df_final['patch_index'].isna().sum()
print(f"NaN patch_index rows: {nan_patches}")

for col in ['team', 'team1_name', 'team2_name']:
    if col in df_final.columns:
        df_final = df_final[~df_final[col].str.contains("Spotlight", na=False)]

df_final = df_final[df_final['patch_index'].notna()]

df_final = df_final[
    df_final.get('team1_pick_1', pd.Series([pd.NA])).notna() &
    df_final.get('team2_pick_1', pd.Series([pd.NA])).notna()
]

df_final = df_final[df_final['Map1_RD'].notna() & df_final['Map2_RD'].notna()]

df_final.to_csv(OUT_PATH, index=False)
print(f"Final output: {len(df_final)} records saved → {OUT_PATH}")

Rows before 7-map gate: 6162
Rows after 7-map gate: 3496
PoolID gaps check: [1, 2]
NaN patch_index rows: 186
Final output: 3264 records saved → /Users/samharwood/Downloads/vlr_patchpool_demo.csv
